In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error

import joblib


In [3]:
# Importing data
df = pd.read_csv('data/labelencoded_data.csv',index_col=0)


In [4]:
# Importing data
df = pd.read_csv('data/labelencoded_data.csv',index_col=0)
# df.drop(['id', 'std', 'sta', 'fltid','arr_iata','dep_iata','ac'], axis=1,inplace=True)


In [5]:
df.head()

,id,datop,fltid,depstn,arrstn,std,sta,status,ac,target,...,flight_time_in_min,flight_distance_in_km,average_flight_speed_km_h,international_flight,year,month,day,season,std_time,sta_time
0,train_id_0,20160103,TU 0712,31,119,20160103103000,20160103125500,0,TU 32AIMN,260.0,...,145.0,1667,690,1,2016,1,3,3,103000,125500
1,train_id_1,20160113,TU 0757,86,119,20160113150500,20160113165500,0,TU 31BIMO,20.0,...,110.0,983,536,1,2016,1,13,3,150500,165500
2,train_id_2,20160116,TU 0214,123,58,20160116041000,20160116064500,0,TU 32AIMN,0.0,...,155.0,1673,648,1,2016,1,16,3,41000,64500
3,train_id_3,20160117,TU 0480,37,91,20160117141000,20160117170000,0,TU 736IOK,0.0,...,170.0,1805,637,1,2016,1,17,3,141000,170000
4,train_id_4,20160117,TU 0338,123,4,20160117143000,20160117155000,0,TU 320IMU,22.0,...,80.0,626,470,1,2016,1,17,3,143000,155000


In [6]:
# df.drop(['id', 'std', 'sta', 'fltid','arr_iata','dep_iata','ac'], axis=1,inplace=True)


In [7]:
df.head().T

,0,1,2,3,4
id,train_id_0,train_id_1,train_id_2,train_id_3,train_id_4
datop,20160103,20160113,20160116,20160117,20160117
fltid,TU 0712,TU 0757,TU 0214,TU 0480,TU 0338
depstn,31,86,123,37,123
arrstn,119,119,58,91,4
std,20160103103000,20160113150500,20160116041000,20160117141000,20160117143000
sta,20160103125500,20160113165500,20160116064500,20160117170000,20160117155000
status,0,0,0,0,0
ac,TU 32AIMN,TU 31BIMO,TU 32AIMN,TU 736IOK,TU 320IMU
target,260.0,20.0,0.0,0.0,22.0


In [8]:
# Splitting data
X = df[['arrstn','depstn','std','sta']]
y = df['target'] 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [12]:
X.head()

,arrstn,depstn,std_time,sta_time,flight_time_in_min,datop
0,119,31,103000,125500,145.0,20160103
1,119,86,150500,165500,110.0,20160113
2,58,123,41000,64500,155.0,20160116
3,91,37,141000,170000,170.0,20160117
4,4,123,143000,155000,80.0,20160117


In [9]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", rmse)


RMSE: 128.43504471511554


/home/pclinux/spiced/data_science/flight_delays_ML/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


RMSE: 128.43504471511554


In [10]:
# Save the model to a file
model_filename = './data/api_test_logistic_regression_model.joblib'
joblib.dump(lr, model_filename)

['./data/api_test_logistic_regression_model.pkl']